In [ ]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Anxiety_disorder"
cohort = "GSE68526"

# Input paths
in_trait_dir = "../../input/GEO/Anxiety_disorder"
in_cohort_dir = "../../input/GEO/Anxiety_disorder/GSE68526"

# Output paths
out_data_file = "../../output/preprocess/Anxiety_disorder/GSE68526.csv"
out_gene_data_file = "../../output/preprocess/Anxiety_disorder/gene_data/GSE68526.csv"
out_clinical_data_file = "../../output/preprocess/Anxiety_disorder/clinical_data/GSE68526.csv"
json_path = "../../output/preprocess/Anxiety_disorder/cohort_info.json"


### Step 1: Initial Data Loading

In [ ]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


### Step 2: Dataset Analysis and Clinical Feature Extraction

In [ ]:
I'll provide a correctly formatted code solution for this step:

```python
# 1. Determine if gene expression data is available
# The background information mentions "Gene expression profiling was carried out on peripheral blood RNA samples"
# This suggests the dataset contains gene expression data, not just miRNA or methylation data
is_gene_available = True

# 2. Identify and convert clinical data

# 2.1. Trait (Anxiety disorder)
# The sample characteristics include "anxiety" scores at index 13
# This is from the Beck Anxiety Inventory mentioned in the description
trait_row = 13

def convert_trait(value):
    if 'missing' in str(value).lower():
        return None
    try:
        # Extract the numeric part after the colon
        parts = value.split(':', 1)
        if len(parts) > 1:
            anxiety_score = float(parts[1].strip())
            # Convert to binary based on a threshold
            # Beck Anxiety Inventory: higher values indicate greater anxiety
            # Using threshold of 2.0 (moderate anxiety)
            return 1 if anxiety_score >= 2.0 else 0
        return None
    except:
        return None

# 2.2. Age
# Age is recorded at index 0
age_row = 0

def convert_age(value):
    try:
        # Extract the numeric part after the colon
        parts = value.split(':', 1)
        if len(parts) > 1:
            age = float(parts[1].strip())
            return age
        return None
    except:
        return None

# 2.3. Gender
# Gender is recorded at index 1 as "female: 0" or "female: 1"
gender_row = 1

def convert_gender(value):
    try:
        # Extract the numeric part after the colon
        parts = value.split(':', 1)
        if len(parts) > 1:
            female = int(parts[1].strip())
            # Convert to standard format where 0=female, 1=male
            # In the data, female=1 means it's a female, female=0 means it's a male
            return 1 - female  # Reverse the coding to match our standard
        return None
    except:
        return None

# 3. Perform initial filtering and save metadata
is_trait_available = trait_row is not None
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Extract clinical features if trait_row is not None
if trait_row is not None:
    # Create a DataFrame from the sample characteristics dictionary provided
    sample_characteristics = {0: ['age (yrs): 79', 'age (yrs): 76', 'age (yrs): 70', 'age (yrs): 65', 'age (yrs): 64', 'age (yrs): 75', 'age (yrs): 66', 'age (yrs): 93', 'age (yrs): 69', 'age (yrs): 67', 'age (yrs): 77', 'age (yrs): 74', 'age (yrs): 73', 'age (yrs): 80', 'age (yrs): 68', 'age (yrs): 83', 'age (yrs): 87', 'age (yrs): 81', 'age (yrs): 84', 'age (yrs): 55', 'age (yrs): 62', 'age (yrs): 58', 'age (yrs): 60', 'age (yrs): 56', 'age (yrs): 86', 'age (yrs): 78', 'age (yrs): 48', 'age (yrs): 82', 'age (yrs): 95', 'age (yrs): 71'], 1: ['female: 0', 'female: 1'], 2: ['black: 0', 'black: 1'], 3: ['hispanic: 0', 'hispanic: 1'], 4: ['bmi: 22.7', 'bmi: 29.1', 'bmi: 25.8', 'bmi: 24.8', 'bmi: 42.1', 'bmi: 29.6', 'bmi: 21.4', 'bmi: 32.7', 'bmi: 30.7', 'bmi: 29.2', 'bmi: 34.0', 'bmi: 44.3', 'bmi: 28.7', 'bmi: 27.4', 'bmi: 30.6', 'bmi: 31.3', 'bmi: 30.0', 'bmi: 25.1', 'bmi: 26.4', 'bmi: 21.6', 'bmi: 18.6', 'bmi: 24.1', 'bmi: 22.9', 'bmi: 28.6', 'bmi: 25.0', 'bmi: 27.5', 'bmi: 25.5', 'bmi: 23.7', 'bmi: 23.0', 'bmi: 28.5'], 5: ['diabcvdcastr: 1', 'diabcvdcastr: 0'], 6: ['ln_hh_income: 16.03', 'ln_hh_income: 15.49', 'ln_hh_income: 15.34', 'ln_hh_income: 15.52', 'ln_hh_income: 16.41', 'ln_hh_income: 14.20', 'ln_hh_income: 18.09', 'ln_hh_income: 13.57', 'ln_hh_income: 16.29', 'ln_hh_income: 14.64', 'ln_hh_income: 15.28', 'ln_hh_income: 15.30', 'ln_hh_income: 15.93', 'ln_hh_income: 15.02', 'ln_hh_income: 16.93', 'ln_hh_income: 14.44', 'ln_hh_income: 16.10', 'ln_hh_income: 14.22', 'ln_hh_income: 13.18', 'ln_hh_income: 14.90', 'ln_hh_income: 12.97', 'ln_hh_income: 16.36', 'ln_hh_income: 16.14', 'ln_hh_income: 14.91', 'ln_hh_income: 13.71', 'ln_hh_income: 15.99', 'ln_hh_income: 13.77', 'ln_hh_income: 15.46', 'ln_hh_income: 13.86', 'ln_hh_income: 16.42'], 7: ['smoke: 1', 'smoke: 0'], 8: ['alcohol: 0', 'alcohol: 1', 'alcohol: missing'], 9: ['loneliness: 1.00', 'loneliness: 2.00', 'loneliness: 1.67', 'loneliness: 1.33', 'loneliness: 2.67', 'loneliness: missing', 'loneliness: 2.33', 'loneliness: 3.00'], 10: ['pwb_purpose: 4.14', 'pwb_purpose: 6.00', 'pwb_purpose: 5.00', 'pwb_purpose: 5.43', 'pwb_purpose: 3.29', 'pwb_purpose: 5.14', 'pwb_purpose: 4.71', 'pwb_purpose: 5.33', 'pwb_purpose: 4.83', 'pwb_purpose: 2.67', 'pwb_purpose: 5.71', 'pw


### Step 3: Dataset Analysis and Clinical Feature Extraction

In [ ]:
I'll implement code that works with the available files without relying on GEOparse.

```python
# Step 1: Analyze the dataset to determine gene expression data availability
import os
import pandas as pd
import json
import numpy as np
import gzip
import re

# First, let's check what files are available in the cohort directory
cohort_files = os.listdir(in_cohort_dir)
print(f"Files in cohort directory: {cohort_files}")

# Let's check if there are matrix files that might contain gene expression data
matrix_files = [f for f in cohort_files if 'matrix' in f.lower()]
print(f"Matrix files: {matrix_files}")

# Load the series matrix file to check for gene expression data
series_matrix_path = os.path.join(in_cohort_dir, matrix_files[0] if matrix_files else cohort_files[0])

# Function to check if file contains gene expression data
def analyze_matrix_file(file_path):
    # Check if the file contains gene expression data by reading header lines
    with gzip.open(file_path, 'rt', encoding='utf-8') as f:
        header_lines = [next(f) for _ in range(100) if '!' in next(f, '')]
    
    # Check if the file contains gene expression data
    is_gene_expression = any(['gene' in line.lower() for line in header_lines]) or \
                        any(['expression' in line.lower() for line in header_lines])
    
    # Check if it's miRNA or methylation only
    is_mirna_only = any(['mirna' in line.lower() for line in header_lines]) and not is_gene_expression
    is_methylation_only = any(['methylation' in line.lower() for line in header_lines]) and not is_gene_expression
    
    return not (is_mirna_only or is_methylation_only)

# Function to parse sample characteristics from series matrix file
def parse_clinical_data(file_path):
    clinical_data = None
    characteristic_lines = []
    sample_ids = []
    
    with gzip.open(file_path, 'rt', encoding='utf-8') as f:
        in_header = True
        for line in f:
            if in_header:
                if line.startswith('!Sample_geo_accession'):
                    sample_ids = line.strip().split('\t')[1:]
                elif line.startswith('!Sample_characteristics_ch'):
                    characteristic_lines.append(line.strip().split('\t')[1:])
                elif line.startswith('!series_matrix_table_begin'):
                    in_header = False
            else:
                break
    
    # Create a dataframe with the clinical data
    if sample_ids and characteristic_lines:
        df_dict = {f'char_{i}': chars for i, chars in enumerate(characteristic_lines)}
        clinical_data = pd.DataFrame(df_dict, index=sample_ids)
    
    return clinical_data

# Determine if gene expression data is available
try:
    is_gene_available = analyze_matrix_file(series_matrix_path)
    print(f"Is gene expression data available: {is_gene_available}")
except Exception as e:
    print(f"Error analyzing gene expression: {e}")
    is_gene_available = False

# Get clinical data from the series matrix file
try:
    clinical_data = parse_clinical_data(series_matrix_path)
    
    if clinical_data is not None:
        print("Clinical data found with shape:", clinical_data.shape)
        print("Sample characteristics:")
        for i, col in enumerate(clinical_data.columns):
            unique_values = clinical_data[col].unique()
            if len(unique_values) < 10:  # Only print if there are few unique values
                print(f"Row {i}: {col} - Unique values: {unique_values}")
            else:
                print(f"Row {i}: {col} - {len(unique_values)} unique values")
    else:
        print("Clinical data not found in the file.")
except Exception as e:
    print(f"Error parsing clinical data: {e}")
    clinical_data = pd.DataFrame()

# Functions to convert trait, age, and gender data
def convert_trait(value):
    if pd.isna(value) or value is None:
        return None
    
    # Extract value after colon if present
    if isinstance(value, str) and ':' in value:
        value = value.split(':', 1)[1].strip()
    
    # Convert to binary (0: control, 1: anxiety disorder)
    value = str(value).lower()
    if 'control' in value or 'healthy' in value or 'normal' in value:
        return 0
    elif 'anxiety' in value or 'ptsd' in value or 'stress' in value or 'disorder' in value:
        return 1
    else:
        return None

def convert_age(value):
    if pd.isna(value) or value is None:
        return None
    
    # Extract value after colon if present
    if isinstance(value, str) and ':' in value:
        value = value.split(':', 1)[1].strip()
    
    # Try to convert to float
    try:
        age = float(value)
        return age
    except (ValueError, TypeError):
        # If age is given as a range (e.g., "25-30"), take the average
        if isinstance(value, str) and '-' in value:
            try:
                parts = value.split('-')
                return (float(parts[0]) + float(parts[1])) / 2
            except (ValueError, IndexError):
                pass
        return None

def convert_gender(value):
    if pd.isna(value) or value is None:
        return None
    
    # Extract value after colon if present
    if isinstance(value, str) and ':' in value:
        value = value.split(':', 1)[1].strip()
    
    # Convert to binary (0: female, 1: male)
    value = str(value).lower()
    if 'female' in value or 'f' == value:
        return 0
    elif 'male' in value or 'm' == value:
        return 1
    else:
        return None

# Based on the analysis, determine which rows contain trait, age, and gender
# Setting default values as None (not available)
trait_row = None
age_row = None
gender_row = None

# Analyze clinical data to find relevant rows
if clinical_data is not None and not clinical_data.empty:
    for i, col in enumerate(clinical_data.columns):
        # Get string representation of column values
        col_str = ' '.join(str(v) for v in clinical_data[col].unique())
        
        # Look for trait indicators
        if ('diagnosis' in col_str.lower() or 'disease' in col_str.lower() or 
            'condition' in col_str.lower() or 'group' in col_str.lower() or 
            'anxiety' in col_str.lower() or 'control' in col_str.lower()):
            # Check if this column could be trait data
            unique_values = clinical_data[col].unique()
            # Verify if the values correspond to trait data (should have at least 2 groups)
            if len(unique_values) > 1:
                trait_row = i
        
        # Look for age indicators
        if 'age' in col_str.lower():
            unique_values = clinical_data[col].unique()
            if len(unique_values) > 1:  # Should have variation in age
                age_row = i
        
        # Look for gender indicators
        if 'gender' in col_str.lower() or 'sex' in col_str.lower():
            unique_values = clinical_data[col].unique()
            if len(unique_values) > 1:  # Should have both males and females
                gender_row = i

# Determine trait data availability
is_trait_available = trait_row is not None

print(f"Trait row: {trait_row}")
print(f"Age row: {age_row}")
print(f"Gender row: {gender_row}")
print(f"Is trait available: {is_trait


### Step 4: Dataset Analysis and Clinical Feature Extraction

In [ ]:
import pandas as pd
import numpy as np
import os
import re
import json
import gzip
import io
from typing import Optional, Callable, Dict, Any, List

def get_feature_data(clinical_df, row_index, feature_name, convert_func):
    """Helper function to extract and convert feature data from a row."""
    feature_series = clinical_df.iloc[row_index].copy()
    feature_series.name = feature_name
    if convert_func is not None:
        feature_series = feature_series.apply(convert_func)
    return pd.DataFrame(feature_series).T

# First, check what files are available in the cohort directory
files = os.listdir(in_cohort_dir)
print(f"Available files: {files}")

# Determine if gene expression data is available
# GEO series matrix files typically contain gene expression data
is_gene_available = any('series_matrix' in f for f in files)

# Extract clinical data from the series matrix file
clinical_data = None
series_matrix_file = [f for f in files if 'series_matrix' in f.lower()][0]
file_path = os.path.join(in_cohort_dir, series_matrix_file)

# Parse the GEO series matrix file to extract sample characteristics
sample_char_lines = []
with gzip.open(file_path, 'rt') as f:
    in_sample_char_section = False
    sample_ids = []
    
    for line in f:
        line = line.strip()
        
        # Extract sample IDs
        if line.startswith('!Sample_geo_accession'):
            sample_ids = line.split('\t')[1:]
            
        # Collect sample characteristics lines
        elif line.startswith('!Sample_characteristics_ch'):
            sample_char_lines.append(line.split('\t')[1:])
            
        # Check if we're done with the characteristics section
        elif line.startswith('!Sample_') and sample_char_lines:
            continue
        elif line.startswith('!series_matrix_table_begin'):
            break

# Create clinical dataframe if sample characteristics were found
if sample_char_lines and sample_ids:
    clinical_data = pd.DataFrame(sample_char_lines, columns=sample_ids)
    print(f"Clinical data shape: {clinical_data.shape}")
    print("First few rows of clinical data:")
    print(clinical_data.head(10))

    # Look for trait (anxiety disorder), age, and gender data in the characteristics
    trait_row = None
    age_row = None
    gender_row = None
    
    # Print unique values for each row to help identify relevant rows
    for i in range(len(clinical_data.index)):
        unique_vals = clinical_data.iloc[i, :].unique()
        print(f"Row {i} unique values: {unique_vals}")
        
        # Check if this row might contain trait data
        row_str = ' '.join(str(val).lower() for val in unique_vals)
        if ('anxiety' in row_str or 'patient' in row_str or 'diagnosis' in row_str or 
            'disorder' in row_str or 'case' in row_str or 'control' in row_str):
            print(f"Potential trait row: {i}")
            trait_row = i
        
        # Check if this row might contain age data
        if 'age' in row_str or 'years' in row_str:
            print(f"Potential age row: {i}")
            age_row = i
        
        # Check if this row might contain gender data
        if 'gender' in row_str or 'sex' in row_str or 'male' in row_str or 'female' in row_str:
            print(f"Potential gender row: {i}")
            gender_row = i

    # Define conversion functions based on observed data patterns
    def convert_trait(value):
        if pd.isnull(value):
            return None
        value_str = str(value).lower()
        
        # Extract value after colon if present
        if ":" in value_str:
            value_str = value_str.split(":", 1)[1].strip()
        
        # Convert to binary: 1 for anxiety, 0 for control/healthy
        if re.search(r'anxi|patient|disorder|case', value_str):
            return 1
        elif re.search(r'control|healthy|normal', value_str):
            return 0
        return None

    def convert_age(value):
        if pd.isnull(value):
            return None
        value_str = str(value).lower()
        
        # Extract value after colon if present
        if ":" in value_str:
            value_str = value_str.split(":", 1)[1].strip()
        
        # Try to extract numeric age value
        age_match = re.search(r'(\d+(?:\.\d+)?)', value_str)
        if age_match:
            return float(age_match.group(1))
        return None

    def convert_gender(value):
        if pd.isnull(value):
            return None
        value_str = str(value).lower()
        
        # Extract value after colon if present
        if ":" in value_str:
            value_str = value_str.split(":", 1)[1].strip()
        
        # Convert to binary: 0 for female, 1 for male
        if re.search(r'female|f\b|woman|women', value_str):
            return 0
        elif re.search(r'male|m\b|man|men', value_str):
            return 1
        return None

    # Check if trait data is available
    is_trait_available = trait_row is not None

    # Save initial metadata
    validate_and_save_cohort_info(
        is_final=False,
        cohort=cohort,
        info_path=json_path,
        is_gene_available=is_gene_available,
        is_trait_available=is_trait_available
    )

    # Extract clinical features if trait data is available
    if is_trait_available:
        print("\nExtracting clinical features...")
        selected_clinical_data = geo_select_clinical_features(
            clinical_df=clinical_data,
            trait=trait,
            trait_row=trait_row,
            convert_trait=convert_trait,
            age_row=age_row,
            convert_age=convert_age if age_row is not None else None,
            gender_row=gender_row,
            convert_gender=convert_gender if gender_row is not None else None
        )
        
        # Preview the selected clinical data
        preview = preview_df(selected_clinical_data)
        print("\nSelected Clinical Data Preview:")
        print(preview)
        
        # Save the clinical data
        os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
        selected_clinical_data.to_csv(out_clinical_data_file)
        print(f"Clinical data saved to {out_clinical_data_file}")
    else:
        print("\nNo suitable trait data found in the clinical information.")
else:
    print("No sample characteristics found in the series matrix file.")
    is_trait_available = False
    
    # Save the negative result
    validate_and_save_cohort_info(
        is_final=False,
        cohort=cohort,
        info_path=json_path,
        is_gene_available=is_gene_available,
        is_trait_available=False
    )


### Step 5: Gene Data Extraction

In [ ]:
# 1. Re-identify the SOFT and matrix files to ensure we have the correct paths
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Extract the gene expression data from the matrix file
gene_data = get_genetic_data(matrix_file)

# 3. Print the first 20 row IDs (gene or probe identifiers)
print("\nFirst 20 gene/probe identifiers:")
print(gene_data.index[:20])

# 4. Print the dimensions of the gene expression data
print(f"\nGene data dimensions: {gene_data.shape[0]} genes × {gene_data.shape[1]} samples")

# Note: we keep is_gene_available as True since we successfully extracted gene expression data
is_gene_available = True


### Step 6: Gene Identifier Review

In [ ]:
# The identifiers in the gene expression data appear to be human gene symbols.
# This can be determined because we see standard gene symbols like A1BG, A2M, AAAS, etc.
# These are official HGNC (HUGO Gene Nomenclature Committee) gene symbols for human genes.

# Since the data is already using human gene symbols, we don't need to perform mapping.
requires_gene_mapping = False

# Printing the result for clarification (though the variable assignment above is sufficient)
print(f"Gene mapping requirement: {requires_gene_mapping}")